# CS524 Project: Flux Balance Analysis for Metabolic Networks

**About this notebook**

The notebook contains two parts:
- In the first part, I will go through and illustrate the problem settings, data processing,
model construction and post-solving analysis. 
- In the second part, I will wrap the whole procedure into a general solver application.
A user can use this to find solution to any organism in a list (yeast, mouse, E. Coli) or
compare performances.

---

## 1. Introduction
Biological systems has numerous interconnected
metabolic (chemical) reactions that convert nutrients into energy, biomass, and other prod-
ucts. Understanding how the reaction ﬂuxes are distributed across a metabolic network
is an important question in biology. Flux Balance Analysis (FBA) is a computational
method to model the reaction ﬂuxes in a metabolic network at steady state.1 It uses a
complete set of all reactions of a cell/organism, and formulate a linear program model to
optimize a biological objective such as growth rate, and hence obtaining a metabolic net-
work model with optimal ﬂux distribution. In this project, I will use the idea of FBA
to construct metabolic models using public data, and then perform subsequent analyses.


## 2. Date processing

### 2.1 The data pipeline
The data we use in this project comes from the BiGG Models [http://bigg.ucsd.edu/](http://bigg.ucsd.edu/).
These are raw data come from experimental measurement of all metabolites (i.e. intermeidate substrates) and 
all reactions and their identities.

The common objective of optimization for simple organisms are defined as "biomass accumulation". This is an idea
similar to taking into some nutrient and grow, and it wants to maximize the growth rate. People have defined
some biomass reaction which transform certain amounts of a set of metabolites into "biomass".
However, for more complex living systems, like human, there is not a simple objective like this. Researchers
are likely to define some objective of interest, like maximizing "something", so a biomass reaction is not contained. In this project, I only 

### Part A. The process of data pipeline and model construction

In [7]:
import requests
import json
import pandas as pd
import os

import gamspy as gp 
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# select one model
NAME = "E. coli (core)"

names_dict = {"E. coli (core)":"e_coli_core",
         "E. coli": "iAF1260",
         "Brewer's yeast (S. cerevisiae)": "iMM904", 
         "House mouse (M. musculus)": "iMM1415"}

def fetch_data(model_name, save_loc=None):
    url = f"http://bigg.ucsd.edu/static/models/{model_name}.json"
    response = requests.get(url)
    if save_loc:
        with open(save_loc, "wb") as f:
            f.write(response.content)
    return response.json()

if os.path.exists(f"data/{names_dict[NAME]}.json"):
    with open(f"data/{names_dict[NAME]}.json", "r") as f:
        data_json = json.load(f)
        print("Loaded from local file.")
else:
    if not os.path.exists("data"):
        os.makedirs("data")
    data_json = fetch_data(names_dict[NAME], f"data/{names_dict[NAME]}.json")
    print("Fetched from web and saved locally.")
    
data_json.keys()

Loaded from local file.


dict_keys(['metabolites', 'reactions', 'genes', 'id', 'compartments', 'version'])

In [32]:

metabolites_df = pd.DataFrame(data_json['metabolites'])
metabolites_df[['id', 'name', 'formula']].head()

,id,name,formula
0,glc__D_e,D-Glucose,C6H12O6
1,gln__L_c,L-Glutamine,C5H10N2O3
2,gln__L_e,L-Glutamine,C5H10N2O3
3,glu__L_c,L-Glutamate,C5H8NO4
4,glu__L_e,L-Glutamate,C5H8NO4


In [48]:
reactions_df = pd.DataFrame(data_json['reactions'])
reactions_df[['id', 'name', 'metabolites', 'lower_bound', 'upper_bound']][15:20]

,id,name,metabolites,lower_bound,upper_bound
15,ATPM,ATP maintenance requirement,"{'adp_c': 1.0, 'atp_c': -1.0, 'h2o_c': -1.0, '...",8.39,1000.0
16,PPCK,Phosphoenolpyruvate carboxykinase,"{'adp_c': 1.0, 'atp_c': -1.0, 'co2_c': 1.0, 'o...",0.00,1000.0
17,ACt2r,Acetate reversible transport via proton symport,"{'ac_c': 1.0, 'ac_e': -1.0, 'h_c': 1.0, 'h_e':...",-1000.00,1000.0
18,PPS,Phosphoenolpyruvate synthase,"{'amp_c': 1.0, 'atp_c': -1.0, 'h2o_c': -1.0, '...",0.00,1000.0
19,ADK1,Adenylate kinase,"{'adp_c': 2.0, 'amp_c': -1.0, 'atp_c': -1.0}",-1000.00,1000.0


In [59]:
reactions_df = pd.DataFrame(data_json['reactions'])

reaction_mat_df = pd.concat([reactions_df[['id', 'name']],   
           pd.DataFrame(reactions_df['metabolites'].to_list()).fillna(0) ],
           axis=1)
# find biomass reaction
biomass_rxn_idx = reaction_mat_df['name'].str.contains('biomass', case=False)
# replace id to "biomass"
reaction_mat_df.at[biomass_rxn_idx.idxmax(), 'id'] = 'biomass'
reaction_mat_df.head()
# get columes other than id name
display(reaction_mat_df[reaction_mat_df.columns.difference(['id', 'name'])].head())

# Convert wide format to long format
reaction_mat_long = pd.melt(
    reaction_mat_df, 
    id_vars=['id'], 
    value_vars=[col for col in reaction_mat_df.columns if col not in ['id', 'name']],
    var_name='metabolite', 
    value_name='coefficient'
)

# Display the result
print(f"Shape: {reaction_mat_long.shape}")
reaction_mat_long.columns = ['rxn', 'metabolite', 'coefficient']
reaction_mat_long.head(5)

,13dpg_c,2pg_c,3pg_c,6pgc_c,6pgl_c,ac_c,ac_e,acald_c,acald_e,accoa_c,...,pyr_e,q8_c,q8h2_c,r5p_c,ru5p__D_c,s7p_c,succ_c,succ_e,succoa_c,xu5p__D_c
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,-1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Shape: (6840, 3)


,rxn,metabolite,coefficient
0,PFK,adp_c,1.0
1,PFL,adp_c,0.0
2,PGI,adp_c,0.0
3,PGK,adp_c,1.0
4,PGL,adp_c,0.0


## 3. The optimization problem

### 3.1 FBA
In this problem setting, we have metabolites $\{m_j\} \ (i=1,2,...M)$, reactions $\{r_i\} \ (j=1,2,...R)$.
Each reaction is consuming and producing some metabolites, with some given chemical ratio, which is called 
the stoichiometric coefficients. We define the **stoichiometric coefficients** $s_{ij}$ is the amount of 
$m_j$ produced by **one unit of forward reaction** $r_i$. We also have some artificial "reactions": (1) exchanges: the reaction that only produce (or its reverse way consumes) a metabolite, which corresponds to
e.g. getting $\mathrm O_2$ from the outside of the cell; and (2) a special "biomass" reaction, which is used 
to define the objective.



In [40]:
gc = gp.Container()

# define sets
metabolite = gp.Set(gc, 'metabolite', description="metabolites", records=metabolites_df['id'])
rxn = gp.Set(gc, 'rxn', description="reactions", records=reaction_mat_df['id'])

# stoichiometric coefficients used for flux balance constraints
stoichiometry = gp.Parameter(gc, 'stoichiometry', description="stoichiometric coefficients",
                     domain=[rxn, metabolite], records=reaction_mat_long)

In [64]:
stoichiometry.records[(stoichiometry.records['rxn']=='biomass') & (stoichiometry.records['value']!=0) ]

,rxn,metabolite,value
24,biomass,adp_c,59.8100
119,biomass,atp_c,-59.8100
214,biomass,f6p_c,-0.0709
404,biomass,h_c,59.8100
499,biomass,accoa_c,-3.7478
594,biomass,coa_c,3.7478
784,biomass,pyr_c,-2.8328
879,biomass,g6p_c,-0.2050
1069,biomass,3pg_c,-1.4960
1354,biomass,h2o_c,-59.8100


In [42]:
print(reaction_mat_df.shape)
print(reaction_mat_df.columns[:10])
reaction_mat_df.iloc[:3, :5]

(95, 74)
Index(['id', 'name', 'adp_c', 'atp_c', 'f6p_c', 'fdp_c', 'h_c', 'accoa_c',
       'coa_c', 'for_c'],
      dtype='object')


,id,name,adp_c,atp_c,f6p_c
0,PFK,Phosphofructokinase,1.0,-1.0,-1.0
1,PFL,Pyruvate formate lyase,0.0,0.0,0.0
2,PGI,Glucose-6-phosphate isomerase,0.0,0.0,1.0
